Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [1]:
import numpy as np
import numpy as np
from tqdm import tqdm
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [3]:
import pandas as pd

# Read input files

In [4]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

CPU times: user 3.55 s, sys: 2.43 s, total: 5.98 s
Wall time: 5.25 s


In [5]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Create submission

In [6]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [7]:
latest_bought_articles = pd.read_csv('/kaggle/input/repurchase4weeks/repurchase4Weeks.csv')

In [8]:
latest_bought_articles = latest_bought_articles.values.tolist()


# Change repurchase to similar items with same group code

In [9]:
# Function to replace article_id with random similar articles
def replace_article_ids(article_ids):
    new_article_ids = []

    for article_id in (article_ids):
        if article_id !=  0:
            product_code = articles.loc[articles['article_id'] == article_id, 'product_code'].values[0]
            similar_articles = articles.loc[articles['product_code'] == product_code, 'article_id'].tolist()

            # Exclude the original article_id from similar articles
            similar_articles = [a for a in similar_articles if a not in article_ids]

            # Shuffle the list of similar articles

            # Take up to 12 random similar articles
            new_article_ids.extend(similar_articles)
    np.random.shuffle(new_article_ids)
    new_article_ids[:min(12, len(new_article_ids))]
    while len(new_article_ids)<12:
        new_article_ids.append(0)
    return new_article_ids
new_last_bought_articles = []
# Replace article_ids in the last_bought_articles list
new_last_bought_articles = [replace_article_ids(article_ids) for article_ids in tqdm(latest_bought_articles)]



100%|██████████| 1371980/1371980 [10:08<00:00, 2255.97it/s]


In [10]:
print(new_last_bought_articles[:3])

[[568601018, 568601044, 568601007, 568601008, 568601006, 568601034, 568601038, 568601033, 568601045, 568601026, 568601019, 568601016, 568601013, 568601012, 568601022, 568601037, 568601023, 568601024, 568601020, 568601030, 568601031, 568601029], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [794321002, 794321001, 794321011, 794321003, 794321004, 794321008, 794321005, 0, 0, 0, 0, 0]]


In [11]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in new_last_bought_articles]
sub.prediction = preds

In [12]:
sub_name = 'similarity_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)